고객 구매 데이터를 사용해서 고객이 주문한 물품이 제 시간에 도착여부(Reached.on.Time_Y.N)예측


In [279]:
!git clone https://github.com/Soyoung-Yoon/bigdata

fatal: destination path 'bigdata' already exists and is not an empty directory.


In [280]:
import pandas as pd
# 이상한 것 확인
df = pd.DataFrame({'rating':['1','2','4','*7','8','*7','3','5','2','*4']})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


In [281]:
#해결 방법
df['rating'] = df['rating'].replace('\*','',regex=True).astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rating  10 non-null     int64
dtypes: int64(1)
memory usage: 208.0 bytes


데이터 불러오기,탐색

In [282]:
#[0] 사용 라이브러리 import
import pandas as pd
#데이터가 많은 경우 모두 출력 안하고 생략해서 출력됨
pd.set_option('display.max_rows',500) #출력할 max row를 지정
pd.set_option('display.max_columns',20) #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format','{:.4f}'.format)

In [283]:
# x_train,x_test 데이터로 생성함
from sklearn.model_selection import train_test_split
dftot = pd.read_csv('bigdata/1st_Train.csv')
display(dftot.head())
dftot.info()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [284]:
x_train,x_test = train_test_split(dftot,test_size=0.4,
                                                 stratify=dftot['Reached.on.Time_Y.N'],random_state=0)
y_train = x_train[['ID','Reached.on.Time_Y.N']]
x_train = x_train.drop(columns='Reached.on.Time_Y.N')
y_test = x_test[['ID','Reached.on.Time_Y.N']]
x_test = x_test.drop(columns='Reached.on.Time_Y.N')

x_train.to_csv('x_train.csv',index=False)
y_train.to_csv('y_train.csv',index=False)
x_test.to_csv('x_test.csv',index=False)
y_test.to_csv('y_test.csv',index=False)

In [285]:
#[1] x_train.csv,y_train.csv,x_train.csv 읽어오기
X = pd.read_csv('x_train.csv')
Y = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')
print(X.shape,Y.shape,X_submission.shape)

(6599, 11) (6599, 2) (4400, 11)


In [286]:
#[2] X,X_submission 합치기 -> dfX로 저장
dfX = pd.concat([X,X_submission],axis=0,ignore_index=True)
print(dfX.shape)
dfX.head()

(10999, 11)


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9902,F,Ship,3,5,214,2,medium,F,6,4578
1,9501,A,Ship,4,2,201,3,medium,F,4,4613
2,6111,A,Ship,4,2,264,3,low,M,10,5137
3,588,F,Ship,3,5,194,2,medium,M,4,3496
4,10806,F,Road,5,5,269,5,medium,F,3,1120


In [287]:
#[3] dfX의 정보 확인
dfX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 945.4+ KB


In [288]:
#[4] 결측치 점검
print(dfX.isna().sum())

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64


In [289]:
#[5] Y값의 분포 확인
temp = Y['Reached.on.Time_Y.N'].value_counts(normalize=True).sort_index()
print(temp)

Reached.on.Time_Y.N
0   0.4032
1   0.5968
Name: proportion, dtype: float64


In [290]:
#[6] X,Y를 합쳐 dfXY를 만듦(양쪽에 ID가 있으므로 merge)
dfXY = pd.merge(dfX,Y)
print(dfXY.shape)

(6599, 12)


In [291]:
#[7] 범주형 'Reached.on.Time_Y.N'의 개수,비율을 표시하는 DataFrame 작성 함수
def make_table (df,feature):
  result = pd.DataFrame()
  temp = df.groupby([feature,'Reached.on.Time_Y.N'])[feature].count()
  result['Count'] = temp

  sList = []
  for k in sorted(df[feature].unique()):
    temp = df.loc[df[feature] == k,'Reached.on.Time_Y.N'].value_counts(normalize=True)
    temp = temp.sort_index()
    sList.append(temp)

  result = result.reset_index()
  result['Rate']  = pd.concat(sList,ignore_index=True)
  return result


In [292]:
#[8] 'Warehouse_block','Reached.on.Time_Y.N' 별 'Warehouse_block'의 개수,비율
temp = make_table(dfXY,'Warehouse_block')
print(temp)

  Warehouse_block  Reached.on.Time_Y.N  Count   Rate
0               A                    0    430 0.3895
1               A                    1    674 0.6105
2               B                    0    454 0.4116
3               B                    1    649 0.5884
4               C                    0    442 0.4077
5               C                    1    642 0.5923
6               D                    0    436 0.4071
7               D                    1    635 0.5929
8               F                    0    899 0.4019
9               F                    1   1338 0.5981


In [293]:
dfXY.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,9902,F,Ship,3,5,214,2,medium,F,6,4578,0
1,9501,A,Ship,4,2,201,3,medium,F,4,4613,0
2,6111,A,Ship,4,2,264,3,low,M,10,5137,1
3,588,F,Ship,3,5,194,2,medium,M,4,3496,1
4,10806,F,Road,5,5,269,5,medium,F,3,1120,0


In [294]:
#[9] 'Mode_of_Shipment','Reached.on.Time_Y.N' 별 'Mode_of_Shipment' 의 개수 비율
temp = make_table(dfXY,'Mode_of_Shipment')
print(temp)

  Mode_of_Shipment  Reached.on.Time_Y.N  Count   Rate
0           Flight                    0    433 0.3987
1           Flight                    1    653 0.6013
2             Road                    0    427 0.4142
3             Road                    1    604 0.5858
4             Ship                    0   1801 0.4018
5             Ship                    1   2681 0.5982


In [295]:
#[10] 'Product_importance','Reached.on.Time_Y.N'별  'Product_importance' 의 개수 비율
temp = make_table(dfXY,'Product_importance')
print(temp)

  Product_importance  Reached.on.Time_Y.N  Count   Rate
0               high                    0    211 0.3670
1               high                    1    364 0.6330
2                low                    0   1288 0.4063
3                low                    1   1882 0.5937
4             medium                    0   1162 0.4071
5             medium                    1   1692 0.5929


In [296]:
#[11] 'Gender','Reached.on.Time_Y.N' 별 'Gender' 의 개수 비율
temp = make_table(dfXY,'Gender')
print(temp)

  Gender  Reached.on.Time_Y.N  Count   Rate
0      F                    0   1341 0.4069
1      F                    1   1955 0.5931
2      M                    0   1320 0.3996
3      M                    1   1983 0.6004


In [297]:
#[12] dfX의 각 컬럼의 값 종류 개수 - 1인 것 제거하기 위해
dfXY.nunique()

,0
ID,6599
Warehouse_block,5
Mode_of_Shipment,3
Customer_care_calls,6
Customer_rating,5
Cost_of_the_Product,215
Prior_purchases,8
Product_importance,3
Gender,2
Discount_offered,65


In [298]:
#[13] Object 타입에 대한 Label Encoding => df_LE
df_LE = dfX.copy()
df_LE['Gender'] = dfX['Gender'].replace(['F','M'],['0','1']).astype('int64')
df_LE['Warehouse_block'] = dfX['Warehouse_block'].replace(['A','B','C','D','F'],['0','1','2','3','4']).astype('int64')
df_LE['Mode_of_Shipment'] = dfX['Mode_of_Shipment'].replace(['Ship','Flight','Road'],['0','1','2']).astype('int64')
df_LE['Product_importance'] = dfX['Product_importance'].replace(['low','medium','high'],['0','1','2']).astype('int64')
df_LE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   10999 non-null  int64
 1   Warehouse_block      10999 non-null  int64
 2   Mode_of_Shipment     10999 non-null  int64
 3   Customer_care_calls  10999 non-null  int64
 4   Customer_rating      10999 non-null  int64
 5   Cost_of_the_Product  10999 non-null  int64
 6   Prior_purchases      10999 non-null  int64
 7   Product_importance   10999 non-null  int64
 8   Gender               10999 non-null  int64
 9   Discount_offered     10999 non-null  int64
 10  Weight_in_gms        10999 non-null  int64
dtypes: int64(11)
memory usage: 945.4 KB


In [299]:
#[14] Object 타입에 대한 One Hot Encoding => df_OH
features = ['Warehouse_block','Mode_of_Shipment','Product_importance']
df_OH = dfX.copy()
df_OH['Gender'] = dfLE['Gender'].replace(['Female','Male'],['0','1'])
df_OH = df_OH.drop(columns = features)

tempLIst = [df_OH]
for f in features:
  s = pd.get_dummies(dfX[f])
  tempLIst.append(s)
df_OH = pd.concat(tempLIst,axis=1)

print(df_OH.shape)
print(df_OH.columns)

#df_OH에서 영향력이 있다고 생각되는 것만 골라서 사용
features = ['ID', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
            'Prior_purchases', 'Discount_offered', 'Weight_in_gms','A',
            'B','Flight', 'Road', 'high']
df_MINI = df_OH[features]

#하나씩 하지 않고 한꺼번에 OneHotEncoding
df_OH = dfX.copy()
df_OH['Gender'] = dfX['Gender'].replace(['Female','Male'],['0','1'])
df_OH = pd.get_dummies(df_OH)
print(df_OH.shape)
print(df_OH.columns)


(10999, 19)
Index(['ID', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Gender', 'Discount_offered', 'Weight_in_gms', 'A',
       'B', 'C', 'D', 'F', 'Flight', 'Road', 'Ship', 'high', 'low', 'medium'],
      dtype='object')
(10999, 20)
Index(['ID', 'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms',
       'Warehouse_block_A', 'Warehouse_block_B', 'Warehouse_block_C',
       'Warehouse_block_D', 'Warehouse_block_F', 'Mode_of_Shipment_Flight',
       'Mode_of_Shipment_Road', 'Mode_of_Shipment_Ship',
       'Product_importance_high', 'Product_importance_low',
       'Product_importance_medium', 'Gender_F', 'Gender_M'],
      dtype='object')


데이터 분리,모델 생성 및 학습

In [300]:
#[16] 사용할 도구 import
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score


In [301]:
#[17] train socre,test score,roc_auc_score 를 반환하는 함수 작성
def get_scores(model,xtrain,xtest,ytrain,ytest):
  A = model.score(xtrain,ytrain)
  B = model.score(xtest,ytest)
  ypred = model.predict_proba(xtest)[:,1]
  C = roc_auc_score(ytest,ypred)
  return '{:.4f} {:.4f} {:.4f}'.format(A,B,C)

In [302]:
#[18] 다양한 모델을 만들고 기능을 출력하는 함수 작성
def make_model(xtrain,xtest,ytrain,ytest):
  model_1 = LogisticRegression().fit(xtrain,ytrain)
  print('model_1',get_scores(model_1,xtrain,xtest,ytrain,ytest))

  for k in range(1,10):
    model_2 = KNeighborsClassifier(k).fit(xtrain,ytrain)
    print('model_2',k,get_scores(model_2,xtrain,xtest,ytrain,ytest))

  #overfitting
  model_3 = DecisionTreeClassifier(random_state=0).fit(xtrain,ytrain)
  print('model_3',get_scores(model_3,xtrain,xtest,ytrain,ytest))
  #overfitting 해결
  for d in range(3,8):
    model_3 = DecisionTreeClassifier(max_depth=d,random_state=0).fit(xtrain,ytrain)
    print('model_3',d,get_scores(model_3,xtrain,xtest,ytrain,ytest))

  #overfitting
  model_4 = RandomForestClassifier(random_state=0).fit(xtrain,ytrain)
  print('model_4',get_scores(model_4,xtrain,xtest,ytrain,ytest))
  #overfitting 해결
  for t in range(3,8):
    model_4 = RandomForestClassifier(max_depth=t,random_state=0).fit(xtrain,ytrain)
    print('model_4',t,get_scores(model_4,xtrain,xtest,ytrain,ytest))

  model_5 = XGBClassifier().fit(xtrain,ytrain)
  print('model_5',get_scores(model_5,xtrain,xtest,ytrain,ytest))


In [303]:
#[19] X를 train용도,submission용도로 나누고 ,Y를 1차원으로 바꿈(MinMaxScaler 적용)
def get_data(dfX,Y):
  X = dfX.drop(columns=['ID'])
  X_use = X.iloc[:6599,:]        # 훈련 데이터 (앞부분)
  X_submission = X.iloc[6599:,:] # 제출용 데이터 (뒷부분)
  Y1 = Y['Reached.on.Time_Y.N']
  scaler = MinMaxScaler()
  X1_use = scaler.fit_transform(X_use)
  X1_submission = scaler.fit_transform(X_submission)
  print(X1_use.shape,X1_submission.shape,Y1.shape)
  return X1_use,X1_submission,Y1

In [304]:
#[20] df_LE 사용
#X,Y 분리하기
X1_use,X1_submission,Y1 = get_data(df_LE,Y)
#train,test 7:3분할, stratify 적용, random_state=0 적용
xtrain1,xtest1,ytrain1,ytest1 = train_test_split(X1_use,Y1,
                                                  test_size=0.3,
                                                  stratify = Y1,
                                                  random_state=0)

(6599, 10) (4400, 10) (6599,)


In [305]:
#다양한 모델 만들어보기
make_model(xtrain1,xtest1,ytrain1,ytest1)

model_1 0.6499 0.6434 0.7230
model_2 1 1.0000 0.6288 0.6111
model_2 2 0.8099 0.6379 0.6680
model_2 3 0.8095 0.6439 0.6939
model_2 4 0.7634 0.6394 0.7001
model_2 5 0.7712 0.6465 0.7065
model_2 6 0.7458 0.6510 0.7156
model_2 7 0.7452 0.6510 0.7206
model_2 8 0.7318 0.6545 0.7214
model_2 9 0.7315 0.6389 0.7165
model_3 1.0000 0.6399 0.6251
model_3 3 0.6776 0.6763 0.7429
model_3 4 0.6820 0.6783 0.7437
model_3 5 0.6893 0.6874 0.7487
model_3 6 0.6939 0.6747 0.7404
model_3 7 0.7077 0.6702 0.7332
model_4 1.0000 0.6540 0.7362
model_4 3 0.6807 0.6732 0.7458
model_4 4 0.6926 0.6803 0.7468
model_4 5 0.7002 0.6803 0.7438
model_4 6 0.7231 0.6717 0.7430
model_4 7 0.7521 0.6692 0.7471
model_5 0.9599 0.6510 0.7357


In [306]:
#[21] df_OH 사용 데이터 섞기 X,Y분리하기
#X,Y 분리하기
X2_use,X2_submission,Y2 = get_data(df_OH,Y)
#rain,test 7:3분할, stratify 적용, random_state=0 적용
xtrain2,xtest2,ytrain2,ytest2 = train_test_split(X2_use,Y2,
                                                 test_size=0.3,
                                                 stratify=Y2,random_state=0)

(6599, 19) (4400, 19) (6599,)


In [307]:
#다양한 모델 만들어보기
make_model(xtrain2,xtest2,ytrain2,ytest2)

model_1 0.6491 0.6364 0.7215
model_2 1 1.0000 0.6136 0.5972
model_2 2 0.8060 0.6076 0.6418
model_2 3 0.8160 0.6126 0.6561
model_2 4 0.7625 0.6015 0.6638
model_2 5 0.7712 0.6197 0.6712
model_2 6 0.7413 0.6131 0.6717
model_2 7 0.7387 0.6141 0.6736
model_2 8 0.7212 0.6157 0.6726
model_2 9 0.7229 0.6237 0.6768
model_3 1.0000 0.6455 0.6287
model_3 3 0.6776 0.6763 0.7429
model_3 4 0.6820 0.6783 0.7437
model_3 5 0.6893 0.6874 0.7487
model_3 6 0.6939 0.6747 0.7404
model_3 7 0.7077 0.6702 0.7332
model_4 1.0000 0.6460 0.7261
model_4 3 0.6623 0.6525 0.7494
model_4 4 0.7027 0.6727 0.7435
model_4 5 0.7179 0.6737 0.7398
model_4 6 0.7339 0.6727 0.7412
model_4 7 0.7627 0.6697 0.7376
model_5 0.9632 0.6480 0.7411


In [308]:
#[22] df_MINI 사용 데이터 섞기 ,X,Y 분리하기
#X,Y 분리하기
X3_use,X3_submission,Y3 = get_data(df_MINI,Y)
#rain,test 7:3분할, stratify 적용, random_state=0 적용
xtrain3,xtest3,ytrain3,ytest3 = train_test_split(X3_use,Y3,
                                                 test_size=0.3,
                                                 stratify=Y3,random_state=0)

(6599, 11) (4400, 11) (6599,)


In [309]:
#다양한 모델 만들어보기
make_model(xtrain3,xtest3,ytrain3,ytest3)

model_1 0.6501 0.6389 0.7221
model_2 1 1.0000 0.6222 0.6101
model_2 2 0.8065 0.6338 0.6654
model_2 3 0.8151 0.6359 0.6973
model_2 4 0.7668 0.6460 0.7038
model_2 5 0.7753 0.6308 0.7074
model_2 6 0.7493 0.6399 0.7110
model_2 7 0.7502 0.6439 0.7166
model_2 8 0.7393 0.6470 0.7176
model_2 9 0.7335 0.6419 0.7185
model_3 1.0000 0.6480 0.6331
model_3 3 0.6776 0.6763 0.7429
model_3 4 0.6820 0.6783 0.7437
model_3 5 0.6893 0.6874 0.7487
model_3 6 0.6939 0.6747 0.7404
model_3 7 0.7077 0.6702 0.7332
model_4 1.0000 0.6535 0.7275
model_4 3 0.6774 0.6717 0.7485
model_4 4 0.6937 0.6818 0.7481
model_4 5 0.7047 0.6818 0.7458
model_4 6 0.7181 0.6727 0.7418
model_4 7 0.7443 0.6717 0.7426
model_5 0.9374 0.6561 0.7322


In [311]:
#[23] 최종적으로 사용할 모델을 선택하여 model로 다시 생성
model = RandomForestClassifier(max_depth=3,random_state=0).fit(xtrain2,ytrain2)
print('final model',get_scores(model,xtrain2,xtest2,ytrain2,ytest2))

final model 0.6623 0.6525 0.7494


제출할 데이터 생성

In [322]:
#[24] test 데이터 (X_submission,X2_submission)에 대한 확률 구하기, 파일로 저장하기
pred = model.predict_proba(X2_submission)[:,1]
submission = pd.DataFrame({'ID':X_submission['ID'],
                           'Reached.on.Time_Y.N':pred})
submission.to_csv('submission.csv',index=False)

In [323]:
#submission 확인
submission.head()

,ID,Reached.on.Time_Y.N
0,7007,0.4998
1,9793,0.4979
2,6593,0.5416
3,10527,0.4680
4,6914,0.5538


In [326]:
ytest=pd.read_csv('y_test.csv')
roc_auc_score(ytest['Reached.on.Time_Y.N'],pred)

0.737722441314554